In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

In [2]:
path_to_sqlite='estate_data.sqlite'
con = sqlite3.connect(path_to_sqlite)

In [3]:
print("Tables in our database:\n")
cursor = con.cursor()
# Getting all tables from sqlite_master
sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""
cursor.execute(sql_query)
sqlite_tables = cursor.fetchall()
sqlite_tables = [*map(lambda table: table[0] ,sqlite_tables)]
print(sqlite_tables)
del sqlite_tables
del sql_query

Tables in our database:

['OFFERS_HOUSES_SALE', 'DESCRIPTION_HOUSES_SALE']


In [4]:
df_description = pd.read_sql('SELECT * FROM DESCRIPTION_HOUSES_SALE', con = con)
df_offers = pd.read_sql('SELECT * FROM OFFERS_HOUSES_SALE', con = con)
df_offers.shape, df_description.shape

((4856, 14), (4853, 36))

In [5]:
# Closing the connection
con.close()

In [6]:
df_offers

,hash_id,price,price_czk,price_czk_unit,price_czk_name,land_area,lat,lon,locality,category_main,category_sub,category_type,date_download,locality_region_id
0,2698797404,5299000,5299000,,Celková cena,1,50.083549,14.434141,Praha,2,40,1,2024-02-02,10
1,3981059148,14595304,14595304,,Celková cena,157,50.147034,14.473924,"U spojů, Praha 8 - Ďáblice",2,37,1,2024-02-02,10
2,2171561036,54980000,54980000,,Celková cena,856,50.107578,14.352712,"V Trníčku, Praha 6 - Dejvice",2,37,1,2024-02-02,10
3,2357572940,19600000,19600000,,Celková cena,961,50.023621,14.443550,"Na Větrově, Praha 4 - Lhotka",2,54,1,2024-02-02,10
4,1286899020,23400000,23400000,,Celková cena,354,50.138809,14.546665,Praha 9 - Kbely,2,37,1,2024-02-02,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4851,2860658268,5857977,5857977,,Celková cena,0,50.100857,13.650740,"Trnková, Senomaty",2,37,1,2024-02-02,11
4852,751513180,19800000,19800000,,Celková cena,0,49.893295,14.327883,"Líšnice, okres Praha-západ",2,37,1,2024-02-02,11
4853,586794316,5199000,5199000,,Celková cena,0,49.948387,15.268164,"Kutná Hora, okres Kutná Hora",2,40,1,2024-02-02,11
4854,1408877900,5299000,5299000,,Celková cena,0,50.411350,14.903184,"Mladá Boleslav, okres Mladá Boleslav",2,40,1,2024-02-02,11


Drop:

- category_type (prodej)
- category_main (domy)
- date_download
- price_czk_unit
- price_czk_name
- land_area - error u Stredoceskeho kraje --> pouzit area_landplot

Co je category_sub?


## Details

Pouzit:
- area_net is what I want I think
- area_landplot

Drop:

- description
- date_year_reconstruction
- area_garden

In [42]:
df_description

,hash_id,description,Vlak,Večerka,Obchod,Bus MHD,Škola,Školka,Lékař,Hospoda,...,building_floor,date_year_reconstruction,area_garden,garage,swimming_pool,area_landplot,ownership,cellar,house_location,pool area
0,2698797404,Nízkoenergetický rodinný dům Classic City je u...,212.0,43.0,550.0,47.0,206.0,341.0,430.0,991.0,...,None,NaN,None,None,NaN,1,None,None,Samostatný,None
1,3981059148,Nový rodinný řadový dům o dispozici 4+kk (161 ...,3213.0,719.0,1989.0,60.0,714.0,411.0,2121.0,1667.0,...,None,NaN,92,None,NaN,157,None,None,Řadový,None
2,2171561036,"Samostatně stojící vila s označením TRJE.B2, s...",1441.0,989.0,2088.0,90.0,756.0,1041.0,1208.0,2146.0,...,None,NaN,707,None,NaN,856,None,None,Samostatný,None
3,2357572940,Nabízíme Vám na prodej dvougenerační rodinný d...,1189.0,228.0,831.0,160.0,75.0,178.0,832.0,645.0,...,2,NaN,None,1,NaN,961,None,None,None,None
4,1286899020,Přímý majitel nabízí moderní nízkoenergetickou...,464.0,104.0,2550.0,54.0,279.0,476.0,3115.0,293.0,...,3,NaN,None,1,NaN,354,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4848,2860658268,Přízemní rodinný dům pro čtyřčlennou rodinu s ...,657.0,347.0,0,351.0,431.0,443.0,0,940.0,...,None,NaN,None,None,NaN,650,None,None,Samostatný,None
4849,751513180,"Nabízíme k prodeji vysoce kvalitní, samostatně...",912.0,1620.0,0,621.0,1569.0,1569.0,0,924.0,...,2 včetně 1 podzemního,NaN,937,1,1.0,1217,None,None,Samostatný,None
4850,586794316,Nízkoenergetický rodinný dům Bungalov je určen...,671.0,91.0,1799.0,100.0,227.0,134.0,0,333.0,...,None,NaN,None,None,NaN,1,None,None,Samostatný,None
4851,1408877900,Nízkoenergetický rodinný dům Classic City je u...,1231.0,73.0,1318.0,35.0,141.0,361.0,1259.0,106.0,...,None,NaN,None,None,NaN,1,None,None,Samostatný,None


TODO: check if the values are correct + decide price/area

In [53]:
df_offers[df_offers["locality_region_id"] == 10]

,hash_id,price,price_czk,price_czk_unit,price_czk_name,land_area,lat,lon,locality,category_main,category_sub,category_type,date_download,locality_region_id
0,2698797404,5299000,5299000,,Celková cena,1,50.083549,14.434141,Praha,2,40,1,2024-02-02,10
1,3981059148,14595304,14595304,,Celková cena,157,50.147034,14.473924,"U spojů, Praha 8 - Ďáblice",2,37,1,2024-02-02,10
2,2171561036,54980000,54980000,,Celková cena,856,50.107578,14.352712,"V Trníčku, Praha 6 - Dejvice",2,37,1,2024-02-02,10
3,2357572940,19600000,19600000,,Celková cena,961,50.023621,14.443550,"Na Větrově, Praha 4 - Lhotka",2,54,1,2024-02-02,10
4,1286899020,23400000,23400000,,Celková cena,354,50.138809,14.546665,Praha 9 - Kbely,2,37,1,2024-02-02,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2310949724,24943000,24943000,,Celková cena,373,50.128590,14.363561,Praha 6 - Praha-Lysolaje,2,37,1,2024-02-02,10
766,562432604,27208000,27208000,,Celková cena,466,50.128056,14.364070,Praha 6 - Praha-Lysolaje,2,37,1,2024-02-02,10
767,3329505884,78000000,78000000,,Celková cena,268,50.121460,14.472025,"Davídkova, Praha 8",2,35,1,2024-02-02,10
768,2615465292,4899000,4899000,,Celková cena,1,50.083549,14.434141,Praha,2,40,1,2024-02-02,10


In [54]:
df_offers[df_offers["locality_region_id"] == 11]

,hash_id,price,price_czk,price_czk_unit,price_czk_name,land_area,lat,lon,locality,category_main,category_sub,category_type,date_download,locality_region_id
770,2376369484,8279000,8279000,,Celková cena,0,50.130562,14.729934,"Nehvizdy, okres Praha-východ",2,37,1,2024-02-02,11
771,3284702540,9200000,9200000,,Celková cena,0,50.046257,15.154190,"Sluneční, Nová Ves I - Ohrada",2,37,1,2024-02-02,11
772,4172670284,12990000,12990000,,Celková cena,0,49.932751,14.511170,"Luční, Psáry",2,37,1,2024-02-02,11
773,844498764,3599000,3599000,,Celková cena,0,50.081573,14.258561,"Hostivice, okres Praha-západ",2,40,1,2024-02-02,11
774,2929321292,5000000,5000000,,Celková cena,0,49.879345,14.349342,"Hvozdnice, okres Praha-západ",2,37,1,2024-02-02,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4851,2860658268,5857977,5857977,,Celková cena,0,50.100857,13.650740,"Trnková, Senomaty",2,37,1,2024-02-02,11
4852,751513180,19800000,19800000,,Celková cena,0,49.893295,14.327883,"Líšnice, okres Praha-západ",2,37,1,2024-02-02,11
4853,586794316,5199000,5199000,,Celková cena,0,49.948387,15.268164,"Kutná Hora, okres Kutná Hora",2,40,1,2024-02-02,11
4854,1408877900,5299000,5299000,,Celková cena,0,50.411350,14.903184,"Mladá Boleslav, okres Mladá Boleslav",2,40,1,2024-02-02,11


Join:

In [43]:
df = df_description.merge(df_offers, on = "hash_id", how = "inner")
df.shape

ValueError: You are trying to merge on object and int64 columns for key 'hash_id'. If you wish to proceed you should use pd.concat

Cleaning:

In [34]:
df_description['energy_efficient_rating']

0      Třída B - Velmi úsporná č. 264/2020 Sb. podle ...
1      Třída B - Velmi úsporná č. 78/2013 Sb. podle v...
2      Třída B - Velmi úsporná č. 264/2020 Sb. podle ...
3      Třída G - Mimořádně nehospodárná č. 78/2013 Sb...
4                                Třída B - Velmi úsporná
                             ...                        
765                              Třída B - Velmi úsporná
766                              Třída B - Velmi úsporná
767                                                 None
768    Třída B - Velmi úsporná č. 264/2020 Sb. podle ...
769    Třída B - Velmi úsporná č. 264/2020 Sb. podle ...
Name: energy_efficient_rating, Length: 770, dtype: object

In [35]:
# 'energy_efficient_rating'
def transformer(value):
    if value != None:
        return value.split()[1]
df_description['energy_efficient_rating'] = df_description['energy_efficient_rating'].apply(lambda row: transformer(row))
df_description['energy_efficient_rating'].value_counts()

energy_efficient_rating
G    334
B    160
C     71
D     45
A     12
E     10
F      3
Name: count, dtype: int64

Save to csv:

In [ ]:
df.to_csv("data.csv")